In [1]:
import os 
import glob
import pandas as pd 
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

outdir = 'results/main/sgls/summary/'
os.makedirs(outdir, exist_ok=True)

In [2]:
sgl_fns = glob.glob('results/main/sgls/*/*/*/*/master.tsv') 
print('There are {} SGL results.'.format(len(sgl_fns)))

There are 106 SGL results.


In [3]:
sgl_fns = glob.glob('results/main/GRCh37/sgls/ldpairs/*/*/*/*/*/script_version/master.tsv') 

In [5]:
df

,chrA,startA,endA,chrB,startB,endB,name,score,strandA,strandB,...,eqtl.rsid,eqtl.old_pos,eqtl.was_converted,gene.start,gene.end,gene.strand,gwas_source,eqtl_source,ge_source,loop_source
0,chr14,69258840,69258841,chr14,68265720,68265721,14:69258841-ENSG00000258759,.,+,+,...,NaN,NaN,NaN,68265721,68266326,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
1,chr14,69258840,69258841,chr14,68283306,68283307,14:69258841-ENSG00000072121,.,+,-,...,rs927292,68792124.0,1.0,68194091,68283307,-,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
2,chr14,69258840,69258841,chr14,68286495,68286496,14:69258841-ENSG00000182185,.,+,+,...,rs927292,68792124.0,1.0,68286496,69196935,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
3,chr14,69258840,69258841,chr14,68580860,68580861,14:69258841-ENSG00000274666,.,+,-,...,NaN,NaN,NaN,68580423,68580861,-,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
4,chr14,69258840,69258841,chr14,68596912,68596913,14:69258841-ENSG00000258837,.,+,-,...,rs927292,68792124.0,1.0,68591721,68596913,-,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,chr18,67532291,67532292,chr18,68079290,68079291,18:67532292-ENSG00000265698,.,+,+,...,NaN,NaN,NaN,68079291,68079627,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
134,chr18,67532291,67532292,chr18,68097466,68097467,18:67532292-ENSG00000214255,.,+,+,...,NaN,NaN,NaN,68097467,68098361,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
135,chr18,67532291,67532292,chr18,68156865,68156866,18:67532292-ENSG00000278971,.,+,+,...,NaN,NaN,NaN,68156866,68158441,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton
136,chr18,67532291,67532292,chr18,68179962,68179963,18:67532292-ENSG00000239748,.,+,+,...,NaN,NaN,NaN,68179963,68180241,+,sgls,ldpairs,eQTL_Catalogue,T1D_34012112_Gaulton


In [4]:
data = []
for fn in sgl_fns:
    
    split_info = fn.split('/')
    
    # get the current json string for the current file  
    gwas_source, eqtl_source, ge_source, loop_source = split_info[3:7]  
    
    df = pd.read_table(fn)
    df.loc[:, 'gwas_source'] = gwas_source
    df.loc[:, 'eqtl_source'] = eqtl_source
    df.loc[:, 'ge_source'] = ge_source
    df.loc[:, 'loop_source'] = loop_source
    df = df.loc[(df.has_fithichip_loop == 1)]
    data.append(df)

AttributeError: 'DataFrame' object has no attribute 'has_fithichip_loop'

In [ ]:
all_data = pd.concat(data)

In [ ]:
def get_washu_coords(sr):
    
    snp_pos = sr.snp_pos
    gene_tss = sr.tss_end
    
    if snp_pos < gene_tss:
        left = snp_pos
        right = gene_tss
    else:
        left = gene_tss
        right = snp_pos

    coord = sr.chrom + ':' + str(left) + '-' + str(right)
    
    return(coord)

In [ ]:
all_data.loc[:, 'washu_coords'] = all_data.apply(get_washu_coords, axis=1)

## Finding SGLs

In [ ]:
sgl_data = all_data.loc[(all_data.is_coloc_pair == 1)]

In [ ]:
sgl_data.columns

In [ ]:
sgl_data

In [ ]:
sgl_data[['sid', 'geneid', 'gwas_source', 'eqtl_source', 'loop_source', 'washu_coords']]

## Investigating colocs without FitHiChIP Loops

In [ ]:
eqtl_data = all_data.loc[(all_data.is_eqtl_pair == 1)]

In [ ]:
eqtl_data

In [ ]:
eqtl_data[['sid', 'geneid', 'gwas_source', 'eqtl_source', 'loop_source']]

In [ ]:
eqtl_data.eqtl_source.unique()